In [97]:
import pandas as pd
from tqdm import tqdm

In [29]:
!pip install fastparquet

## 원본 데이터에서 재료만 추출

원본 만개의 레시피.csv 파일을 불러와서 재료만 추출합니다.

In [98]:
try:
    path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\만개의 레시피.csv'
except:
    path = '/content/drive/MyDrive/CP1/만개의 레시피.csv'
ori_df = pd.read_csv(path, encoding='cp949')
ori_df = ori_df.dropna()
ori_df = ori_df.reset_index(drop=True)
ori_df.shape

(114512, 18)

In [99]:
ori_copy = ori_df['CKG_MTRL_CN'].copy()
ori_copy = ori_copy.dropna()
ori_copy

0         [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
1                  [재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
2         [재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...
3         [재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...
4         [재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...
                                ...                        
114507    [재료] 양배추 1/4통| 양파 1/2개| 편마늘 1숟가락| 식용유 적당히| 물 2...
114508    [재료] 박력분 200g| 버터 70g| 크림치즈 70g| 설탕 100g| 계란 1...
114509    [재료] 포항초 300g| 건다시마 작은크기4~5장| 다시멸치 1줌| 홍고추 1개|...
114510    [재료] 식빵 4장| 감자 2개| 양파 1/4개| 피망 1/4개| 계란 2개| 치즈...
114511    [재료] 배추잎 8~10장| 홍고추 1개| 계란 2개 [양념] 부침가루(메밀) 5큰...
Name: CKG_MTRL_CN, Length: 114512, dtype: object

#### 소괄호를 제거하는 함수

In [100]:
from collections import deque
def remove_small_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '(':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ')':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

#### 대괄호를 제거하는 함수

In [101]:
from collections import deque
def remove_big_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '[':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ']':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

전체 데이터를 돌면서 대괄호를 파악하여 '[양념]'을 포함하여 뒷 부분을 제거합니다.  
데이터의 모든 '[재료]'를 포함한 모든 대괄호를 제거합니다.

In [102]:
for i in tqdm(range(len(ori_copy))):
    temp = []
    for j in range(len(ori_copy[i])):
        if ori_copy[i][j] =='[':
            temp.append(j)
    end_index = len(ori_copy[i])
    if len(temp) > 1:
        end_index = temp[1]
    ingre = ori_copy[i][:end_index]
    ingre = remove_big_brackets(ingre).strip()
    ori_copy[i] = ingre

100%|██████████| 114512/114512 [00:40<00:00, 2843.60it/s]


새로운 컬럼을 생성해줍니다.

In [103]:
ori_df['only_ingred'] = None
ori_df['only_ingred']

0         None
1         None
2         None
3         None
4         None
          ... 
114507    None
114508    None
114509    None
114510    None
114511    None
Name: only_ingred, Length: 114512, dtype: object

'|'기준으로 문장을 나누고, 나누어진 리스트에서 문자를 추출하여 양 옆 공백을 제거해준뒤  
문자에 포함되어 있는 소괄호를 제거해줍니다.

다시 공백을 기준으로 문자를 나눈 후에  
생성된 리스트의 길이가 3개가 이상이라면 마지막(단위부분)을 제외하고 추출합니다.  
만약 3개 이하라면 앞에 있는 데이터만 추출합니다.

In [104]:
#한쪽만 존재하는 소괄호를 제거하는 함수
def remove_one_bracket(word):
    if word.find('(') > -1:
        idx = word.find('(')
        return word[:idx]
    if word.find(')') > -1:
        idx = word.find(')')
        return word[:idx]
    return word

In [105]:
word = '소금(asd'
remove_one_bracket(word)

'소금'

In [106]:
for i in tqdm(range(len(ori_copy))):
    lst =[]
    for word in ori_copy[i].split('|'):
        word = word.strip()
        word = remove_small_brackets(word)
        word = remove_one_bracket(word)
        li = word.split()
        if li != []:
            if len(li) < 3:
                lst.append(li[0])
            else:
                string = ' '
                lst.append(string.join(li[:-1]))
    ori_df['only_ingred'][i] = lst

  0%|          | 0/114512 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/3544315013.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ori_df['only_ingred'][i] = lst
100%|██████████| 114512/114512 [02:20<00:00, 812.46it/s]


In [107]:
ori_df[['CKG_MTRL_CN','only_ingred']].head()

,CKG_MTRL_CN,only_ingred
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,"[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]"
2,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루]"
3,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,[파스타면]
4,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


어떤 재료들이 csv파일에 존재하는지 확인합니다.

In [108]:
ingre_list = []
for i in range(len(ori_df['only_ingred'])):
    for word in ori_df['only_ingred'][i]:
        ingre_list.append(word)

어떤 재료가 가장 많이 등장했는지 확인합니다.

In [109]:
from collections import Counter
counter = Counter(ingre_list)
counter.most_common(10)

[('양파', 31241),
 ('대파', 19334),
 ('소금', 18672),
 ('물', 13413),
 ('당근', 13313),
 ('설탕', 12938),
 ('계란', 11724),
 ('청양고추', 10521),
 ('참기름', 8904),
 ('달걀', 8749)]

## 크롤링으로 모은 재료 데이터 처리

In [110]:
try:
    path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\ingredient.parquet'
except:
    path = '/content/drive/MyDrive/CP1/ingredient.parquet'
data_df = pd.read_parquet(path, engine='fastparquet')
data_df.shape

(114512, 2)

위에서 작업한 원본 데이터에서 재료만 추출한 데이터와 합치겠습니다.

In [111]:
data_df = pd.concat([data_df, ori_df['only_ingred']], axis=1)
data_df.sample(10)

,id,ingredient,only_ingred
11330,4015891,"[[건미역, 1웅큼], [오이, 1/2개], [청량고추, 1개], [송송썰은 파, ...","[건미역, 오이, 청량고추, 송송썰은 파, 당근]"
6226,1937877,"[[화이트 새우, 350g], [콩나물, 2줌(200g)], [미나리, 1줌(100...","[화이트 새우, 콩나물, 미나리, 양파, 쪽파, 청고추, 홍고추, 표고버섯, 식용유..."
7646,2382561,[[시판떡갈비 ...,"[시판떡갈비, 피자치즈, 양파, 또띠아, 고구마, 우유, 파슬리가루]"
100174,6934338,"[[데친 죽순, 250g], [맥시코 고추, 3-4개], [맛간장, 1/4컵], [...","[데친 죽순, 맥시코 고추]"
74890,6900982,"[[우엉 채, 304g], [조림용 간장, 5수저], [맛술, 1수저], [식초, ...",[우엉 채]
60327,6882974,[[머핀 ...,"[머핀 7개, 수분율 100%, 우리밀 통밀, 계란, 바닐라 익스트랙, 소금, 포도..."
83389,6912392,"[[둥근호박, 1개], [포도씨유, 4T], [새우젓, 2T], [버섯파우더 ...",[둥근호박]
25906,6834870,"[[애호박, 2조각], [대파, 1/8개], [계란, 1개], [양파, 1/4쪽],...","[애호박, 대파, 계란, 양파, 양송이, 마늘, 식은밥, 버터]"
10243,3259614,"[[꽈리고추, 180g], [밀가루, 2큰술], [물, 약간], [간장, 3큰술],...","[꽈리고추, 밀가루, 물]"
82779,6911641,[[파스타 ...,"[파스타, 양파, 프랑크소시지, 다진마늘, 면수]"


데이터를 수집할 때 오류가 나는 데이터를 빈 리스트([ ])로 수집했습니다.

ingredient컬럼에 빈 리스트([ ])가 몇개 있는지 확인하겠습니다.

In [112]:
empty_index = []
for i in range(len(data_df)):
  if data_df['ingredient'][i] == []:
    empty_index.append(i)
len(empty_index)

536

536개의 빈 리스트([ ])를 확인할 수 있습니다.  
모두 제거하겠습니다.

In [113]:
data_df = data_df.drop(empty_index).reset_index(drop=True)
data_df.shape

(113976, 3)

In [114]:
data_df['ingredient']

0         [[어묵, 2개], [김밥용김, 3장], [당면, 1움큼], [양파, 1/2개], ...
1         [[현미, 4컵], [찹쌀, 2컵], [호두, 50g], [물, 1/2컵], [소금...
2         [[북어포, 1마리], [찹쌀가루, 1C], [간장, 2T], [설탕, 1T], [...
3         [[파스타면], [토마토, 1개], [토마토 페이스트, 3T], [양파, 1/2개]...
4         [[건표고버섯, 9개], [오이, 1/2개], [당근, 1/2개], [양파, 1/2...
                                ...                        
113971    [[양배추, 1/4통], [양파, 1/2개], [편마늘, 1숟가락], [식용유, 적...
113972    [[박력분, 200g], [버터, 70g], [크림치즈, 70g], [설탕, 100...
113973    [[포항초, 300g], [건다시마 작은크기, 4~5장], [다시멸치, 1줌], [...
113974    [[식빵, 4장], [감자, 2개], [양파, 1/4개], [피망, 1/4개], [...
113975    [[배추잎, 8~10장], [홍고추, 1개], [계란, 2개], [부침가루(메밀),...
Name: ingredient, Length: 113976, dtype: object

In [115]:
data_df.sample(3)

,id,ingredient,only_ingred
60188,6883343,"[[신김치, 1포기], [올리브유, 조금], [설탕, 3T], [올리고당, 2T],...",[신김치]
78028,6905778,"[[깻잎, 20~30장], [부침가루, 1컵], [달걀, 2알], [간돼지고기, 1...","[깻잎, 부침가루, 달걀]"
69356,6894829,"[[감자, 7-8알], [올리브오일, 약간], [소금과허브잎 ...","[감자, 올리브오일, 소금과허브잎, 파마산치즈가루, 케첩]"


In [116]:
for i in range(len(data_df[:10])):
  print(data_df['ingredient'][i])

[['어묵', '2개'], ['김밥용김', '3장'], ['당면', '1움큼'], ['양파', '1/2개'], ['당근', '1/2개'], ['깻잎', '6장'], ['튀김가루', '1컵'], ['올리브유', '적당량'], ['간장', '1T'], ['참기름', '1T']]
[['현미', '4컵'], ['찹쌀', '2컵'], ['호두', '50g'], ['물', '1/2컵'], ['소금', '약간']]
[['북어포', '1마리'], ['찹쌀가루', '1C'], ['간장', '2T'], ['설탕', '1T'], ['물', '1T'], ['다진파', '1T'], ['다진마늘', '1T'], ['참기름', '1T'], ['깨소금', '1T'], ['후춧가루', '약간']]
[['파스타면'], ['토마토', '1개'], ['토마토 페이스트', '3T'], ['양파', '1/2개'], ['다진마늘', '1T'], ['피망', '1/2개'], ['올리브유', '3T']]
[['건표고버섯', '9개'], ['오이', '1/2개'], ['당근', '1/2개'], ['양파', '1/2개'], ['사과', '1/2쪽'], ['그외의                                                                                    야채'], ['과일'], ['녹말가루', '2C'], ['물', '1C'], ['계란 노른자', '1개'], ['물', '2C'], ['설탕', '1/2C'], ['식초', '3T'], ['간장', '1T'], ['녹말물', '2T']]
[['닭', '1마리'], ['통마늘', '3개'], ['다진마늘', '1T'], ['화이트와인또는우유'], ['올리브유또는버터'], ['마늘솔트또는소금'], ['파인애플', '2쪽'], ['마요네즈', '2T'], ['머스터드', '1T'], ['식초', '1/2T(또는 레몬즙)'], ['꿀', '1T']]
[['양파', '1개'], ['빵가루'], ['달걀 노른자',

## 공백이 84(엄청~~긴) 데이터, 단위가 없는 데이터, [](빈리스트) 처리

```python

[['쌈배추 작은것                                                                                    4~5장'], ['달래'], ['김'], []]
```

- 공백이 긴 데이터는 공백을 기준으로 나누고
- 단위가 없는 데이터는 1을 추가해줍니다.
- 빈리스트([]) 데이터는 제거해줍니다.


In [117]:
lists = [['쌈배추 작은것                                                                                    4~5장'], ['달래'], ['김'], []]

In [118]:
def space_nounits(data):
  space = ' '*84 #84는 공백의 길이
  lists = data.copy()
  if [] in lists:
    lists.remove([])
  for i in range(len(lists)):
    if len(lists[i]) == 1 and space in lists[i][0] :
      lists[i] = lists[i][0].split(space)
    elif len(lists[i]) == 1 and lists[i] != []:
      lists[i].append(str('1단위'))
  return lists

In [119]:
print(lists)
print('-'*100)
space_nounits(lists)

[['쌈배추 작은것                                                                                    4~5장'], ['달래'], ['김'], []]
----------------------------------------------------------------------------------------------------


[['쌈배추 작은것', '4~5장'], ['달래', '1단위'], ['김', '1단위']]

In [120]:
lists2 = data_df['ingredient'][68].copy()
lists2

[['박력분', '285그램'],
 ['버터', '88그램'],
 ['계란', '3개'],
 ['설탕', '180그램'],
 ['애호박', '160그램'],
 ['우유', '3~5큰술'],
 ['사워크림', '250그램'],
 ['베이킹 파우더', '4그램'],
 ['소금', '4그램'],
 ['바닐라오일', '약간'],
 [],
 ['박력                                                                                    80그램'],
 ['버터', '40그램'],
 ['설탕', '40그램']]

In [121]:
space_nounits(lists2)

[['박력분', '285그램'],
 ['버터', '88그램'],
 ['계란', '3개'],
 ['설탕', '180그램'],
 ['애호박', '160그램'],
 ['우유', '3~5큰술'],
 ['사워크림', '250그램'],
 ['베이킹 파우더', '4그램'],
 ['소금', '4그램'],
 ['바닐라오일', '약간'],
 ['박력', '80그램'],
 ['버터', '40그램'],
 ['설탕', '40그램']]

In [123]:
copy_data =data_df.copy()
copy_data.head()

,id,ingredient,only_ingred
0,128671,"[[어묵, 2개], [김밥용김, 3장], [당면, 1움큼], [양파, 1/2개], ...","[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,131871,"[[현미, 4컵], [찹쌀, 2컵], [호두, 50g], [물, 1/2컵], [소금...","[현미, 찹쌀, 호두, 물, 소금]"
2,139247,"[[북어포, 1마리], [찹쌀가루, 1C], [간장, 2T], [설탕, 1T], [...","[북어포, 찹쌀가루]"
3,149207,"[[파스타면], [토마토, 1개], [토마토 페이스트, 3T], [양파, 1/2개]...",[파스타면]
4,151148,"[[건표고버섯, 9개], [오이, 1/2개], [당근, 1/2개], [양파, 1/2...","[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


In [124]:
#test 6분 정도 소요
from tqdm import tqdm
for i in tqdm(range(len(copy_data['ingredient']))):
  try : 
    copy_data['ingredient'][i] = space_nounits(copy_data['ingredient'][i])
  except:
    print(i)

  0%|          | 0/113976 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/353184748.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = space_nounits(copy_data['ingredient'][i])
100%|██████████| 113976/113976 [03:48<00:00, 499.64it/s]


In [125]:
#랜덤 인덱스 추출하여 확인
random_index = copy_data.sample(10).index
for index in random_index:
  print(copy_data['ingredient'][index])

[['감자', '2알'], ['당근', '1/4개'], ['식용유', '1단위'], ['소금', '1단위'], ['후추', '1단위'], ['면보', '1단위']]
[['톳', '150g'], ['맛살', '3줄'], ['오이', '1/2개'], ['청양고추', '1개'], ['새우', '3마리'], ['레몬슬라이스', '4장'], ['간마늘', '1큰술'], ['쪽파', '2뿌리'], ['2배식초', '2큰술'], ['물', '3큰술'], ['맛소금', '2/3티스푼'], ['설탕', '1.5큰술'], ['홀그레인머스타드', '2/3큰술']]
[['원칙을 지키는 비엔나', '12개'], ['떡', '12개'], ['고추장', '1T'], ['설탕', '2T'], ['케찹', '3T'], ['올리고당', '1T'], ['간장', '1T']]
[['파인애플', '1'], ['식초', '1'], ['설탕', '1']]
[['새송이버섯', '2개'], ['계란', '2개'], ['부침가루', '1/2컵'], ['소금', '2꼬집'], ['야채', '취향대로']]
[['사각어묵', '2장'], ['꽈리고추', '100g'], ['간장', '1'], ['맛술', '1'], ['다진 마늘', '1'], ['올리고 당', '1'], ['참기름', '0.5'], ['통깨', '0.5'], ['후춧가루', '적당량']]
[['버터', '220g'], ['밀가루', '360ml'], ['슈가파우더', '90ml'], ['베이킹파우더', '1tsp'], ['전분', '80ml'], ['바닐라에센스', '조금']]
[['서리태', '2~3컵']]
[['김치', '1/2포기'], ['양파', '1/2개'], ['어묵', '1장'], ['새송이버섯', '1개'], ['죽순', '1개'], ['냉동새우', '10마리'], ['낙지', '2마리'], ['피망', '1/2개'], ['후춧가루', '1/5작은술'], ['참기름', '1/2큰술'], ['깨소금', '1큰술'], ['설탕', '

## 소괄호() 제거

In [126]:
from collections import deque
def remove_small_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '(':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ')':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

In [127]:
word = '약간 (적당히)'
remove_small_brackets(word)

'약간 '

문자에 들어있는 소괄호를 제거하겠습니다.

In [128]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        if copy_data['ingredient'][i][j] != []:
            copy_data['ingredient'][i][j][0] = remove_small_brackets(copy_data['ingredient'][i][j][0])
            copy_data['ingredient'][i][j][1] = remove_small_brackets(copy_data['ingredient'][i][j][1])

100%|██████████| 113976/113976 [00:31<00:00, 3581.07it/s]


### 재료부분 소괄호 확인

In [129]:
count = 0
ingredient_list = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        if data != []:
           if '(' in str(data[0]) or ')' in str(data[0]):
                ingredient_list.append(data[0])
print(len(ingredient_list))

100%|██████████| 113976/113976 [00:06<00:00, 16991.91it/s]

701


In [130]:
for i in range(len(ingredient_list)):
    word = ingredient_list[i]
    print(word)

슈가파우더(or
호떡믹스 1봉지(프리믹스
호떡믹스 (잼믹스
계란(중_
명란젓(없어도
파 1토막(
두부 1모(
야채믹스(각종 야채
다크초콜릿 (중탕 처리
발로나코코아가루 적당량 (올리고당 대신
빵(식빵이나
칼라별 피망 약간씩(
오이(
배(
정향 약간(
피망 약간(
딸기즙(딸기
닭(
꿩(
참치액 조금 (없으면
고추장 육회 양념장(
물(
매실액 2 (대체 물엿
무(
당근(
다시육수(
물(
오레오 비스킷(크림제거된것
멸치육수(
쌀엿(물엿
레몬(식초
포도씨유 약간 (틀
삼겹살)
밤통조림(밤
타임가루(없으면
칠리소스(없어도
미나리 약간(
고구마중간 사이즈 1개)
호떡믹스 (잼믹스
버터 110g 정도(
쌀가루 (습식-3시간이상 불린쌀을 방앗간에서 소금만 넣
(다시마
식빵(두께
쌀(식구수
다시마(채썬
식물성오일(식용유
육수치킨육수)
2차 발효 후 반죽에 바를 계란물 (노른자
송편색(쑥
백년초가루)
송편소(녹두
유자청)
호떡믹스 (잼믹스
맛살(또는햄
참치)
참기름(또는 고추씨
호박씨)
데미글라스 소스 2T(또는 굴소스
맛타리버섯(
호떡믹스 (잼믹스
달걀물(바르는
피자치즈 약간(혹은
호떡믹스 (잼믹스
매실청(설탕
당근中(
초록색파프리카中(
재래간장(재래간장이 없으시면 간장을 조금 더 넣어 주세
초간장 (진간장
호떡믹스 (잼믹스
마요네즈(마요네즈를 사용할 경우 손에 조금 질척거림이
설탕(설탕을 넣으실 경우 물 1큰술을 더 넣어 주세요.
호떡믹스 (잼믹스
스리라차 칠리소스입니다. 약간 매운편입니다.)
쇠고기사태 한근(
바질 약간 (일반 소고기패티
대파 흰대(
아몬드(생략
너트메그가루(생략 가
영양부추 한줌(
파프리카 (
빼빼로 스틱 32개 (
오이피클(익은
리코타치즈 블럭(까망베르나 브리치즈 슬라이스 치즈대체가
키위(골드키위
그린키위)
뒷다리살)
소고기밑간(청주
연유(또는
설탕)
에스프레소 40g(혹은 커피가루
다시마(5X
육수(다시마와
피망(오이나
로메인(상추 대체
비트무(또는
항암배춧잎(또는
채소(항암 배춧잎/비트잎/붉은
호떡믹스 (잼믹스
우유(얼린것) 적당량(
밥(
화이트 초

위처럼 괄호가 한쪽씩 있는 데이터가 존재하는 것을 확인할 수 있습니다.

### 단위부분 소괄호 확인

In [131]:
unit_list = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        if data != []:
           if '(' in str(data[1]) or ')' in str(data[1]):
                unit_list.append(data[1])
print(len(unit_list))

100%|██████████| 113976/113976 [00:06<00:00, 17168.64it/s]

706


In [132]:
for i in range(len(unit_list)):
    word = unit_list[i]
    print(word)

계피가루)
150그램(럼주1/2컵
무방)
1.5~2cm)
깍뚝썬것)
50g(설탕으로 대체하려면 입맛에 맞게
2리터(멸치와 다시마를 듬뿍 넣어 우려
2리터(멸치와 다시마를 듬뿍 넣어 우려
된)
1줌(기름기가 적은 앞다리살을 이용했어
3~4컵(멸치몇마리 준비해서 다신물 만드시
생략)
모닝빵등)
40g)
80g)1/2개
83g)1/4개
2~3컵(or 멸치몇마리 준비해서 만든 다
5알 정도)
25g)
1/3술(생략가능-넣으면 더 맛나지만 안넣
1술(전 달달하게 먹는걸 좋아해서 양
1개)
480g)1/2마리
2개(크기에 따라 차이가 있을수 있어요
350g)1/2마리
진간장)
5cm)
1큰술(포도씨유나 카놀라유나 식용유나
300g 기준)
2컵(물 3컵+멸치6마리+다시마 5X5
500ml)2.5컵
1작은술(구연산이 없을때는 식초 2큰술
1+1/2큰술)
5마리( 칵테일새우 이용하면 더 좋아요
4센티)1/2토막
4센티)1/2토막
47g(저는 플레인요플레 두시간 물빼서
300ml)1.5컵
1.5L)7.5컵
1/4컵)50ml
1큰술)1/3개
코팅용)
1개(클수록 국물이 더 진하고 맛있어요
12개)
2큰술(반죽에 맞춰서 조금씩 알아서 넣
생략)
되요)
1개(가로19.8X 세로12 X 높이4
50g)
450g짜리 버터 눈대중으로1/4)
1작은술(또는 캡사이신 매운맛 소스0.
2cm)
만큼)
1캔((200g)
1개(약 세로33cm * 가로 22cm
1망(껍질벗기고 씻은 후 무게2.5kg
것)
+물)
기름)
1T)
100g)1/2팩
용)
100g)
1/2큰술)1큰술
50g)1/2개
30g)1/2개
약간)
1개(길이가 12cm 정도의 통통한 고
50g(생크림이 없으시면 그냥 우유를 넣
607g)
3~4개분) - 이것중 하나만 사용할꺼랍니
5큰술(강력한 매운맛을 원하시면2-3큰
5cm)
가능)
15g(달게 드실분들은 설탕량을 더 늘려
1스푼+ 물1/3컵)
50g)
34g~취향껏)
0.5~0.8mm 두께 10~15cm 길이)
배추김치)
1/2토막(두께 2cm×지름 5cm 정도의
10ml(섞은

단위부분 또한 소괄호가 한쪽씩 들어가 있는 데이터가 존재합니다.

### 한쪽만 있는 소괄호 제거

괄호가 한쪽밖에 없는 뒷부분을 제거합니다.

In [133]:
test_sample = [
    [['어묵', '2개'],['김밥용김', '3장'],['당(면', '1움큼'], ['양파', '1/2개'],['당근', '1/2개'], ['깻잎', '6장'],['튀김가루', '1)컵'], ['올리(브유', '적당량'], ['간장', '1T'],['참기)름', '1T']],
    [['현미', '4컵'], ['찹)쌀', '2컵'], ['호두', '50g'], ['물', '1/2(컵'], ['소(금', '약간']]
]
for i in range(len(test_sample)):
    lst = []
    for j in range(len(test_sample[i])):
        data = test_sample[i][j]
        for k in range(len(data)):
            if data[k].find('(') > -1:
                idx = data[k].find('(')
                data[k] = data[k][:idx]
            if data[k].find(')') > -1:
                idx = data[k].find(')')
                data[k] = data[k][:idx]
            
test_sample      

[[['어묵', '2개'],
  ['김밥용김', '3장'],
  ['당', '1움큼'],
  ['양파', '1/2개'],
  ['당근', '1/2개'],
  ['깻잎', '6장'],
  ['튀김가루', '1'],
  ['올리', '적당량'],
  ['간장', '1T'],
  ['참기', '1T']],
 [['현미', '4컵'], ['찹', '2컵'], ['호두', '50g'], ['물', '1/2'], ['소', '약간']]]

In [134]:
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        for k in range(len(data)):
            if data[k].find('(') > -1:
                idx = data[k].find('(')
                data[k] = data[k][:idx]
            if data[k].find(')') > -1:
                idx = data[k].find(')')
                data[k] = data[k][:idx]

100%|██████████| 113976/113976 [00:07<00:00, 15162.35it/s]


In [135]:
lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for j in range(len(copy_data['ingredient'][i])):
        data = copy_data['ingredient'][i][j]
        if data != []:
            if ('(' in data[0]) or(')' in data[0]) or ('(' in data[1]) or (')' in data[1]):
                lst.append(data)
print(len(lst))

100%|██████████| 113976/113976 [00:06<00:00, 17670.40it/s]

0


괄호가 없는 것을 확인할 수 있습니다.

마지막으로 데이터에 소괄호가 있는지 확인해보겠습니다.

In [136]:
#랜덤 인덱스 추출하여 확인
random_index = copy_data.sample(10).index
for index in random_index:
  print(copy_data['ingredient'][index])

[['찹쌀', '3컵'], ['맵쌀', '3컵'], ['팥', '2컵'], ['수수', '1컵'], ['조', '1컵'], ['밤', '20개'], ['소금', '1/2큰술']]
[['로메인', '2포기'], ['파마산치즈가루', '3T'], ['레드와인식초', '2T'], ['엑스트라버진 올리브오일', '1/3cup'], ['간마늘', '1t'], ['디죤머스타드', '2t'], ['소금', '2꼬집'], ['후추가루', '2꼬집']]
[['식빵', '4장'], ['슬라이스햄', '4장'], ['체다치즈', '2장'], ['딸기쨈', '적당량'], ['달걀', '2개'], ['우유', '1/2종이컵'], ['버터', '2숟가락']]
[['말린무청시래기', '적당량'], ['밤', '5톨'], ['은행', '15알'], ['집간장', '2큰술'], ['고추가루', '2큰술'], ['양파', '1/4개'], ['파', '1/2뿌리'], ['참기름', '조금'], ['참깨', '약간'], ['매실청', '1스푼']]
[['참외', '1단위']]
[['꽃게', '1단위'], ['콩나물', '1단위'], ['미나리', '1단위'], ['부추', '1단위'], ['레몬', '1단위']]
[['애호박', '1개'], ['양파', '1/2개'], ['청양고추', '1개'], ['팽이버섯', '1움큼'], ['홍고추', '1개'], ['식용류', '적당량'], ['참기름', '약간'], ['부침가루', '170ml'], ['물', '200ml'], ['소금', '1티스푼']]
[['통삼겹살', '3근'], ['다진마늘', '1큰술'], ['생강술', '2큰술'], ['허브솔트', '적당히']]
[['깐밤', '500g'], ['물', '400'], ['흑설탕갈색/백설탕 대체가능', '250'], ['물', '150']]
[['LA갈비', '2.204Kg'], ['소주', '1컵'], ['양파', '2개'], ['사과', '2개'], ['마늘', '7알'], ['간장', '7

## 양념부분제거(재료만 추출해서)

크롤링으로 모은 재료 데이터에서 양념부분을 제외한 실제 재료 데이터만 추출하겠습니다.

In [141]:
all_lst = []
for i in tqdm(range(len(copy_data))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if len(data) == 0:
            pass
        else:
            if len(copy_data['only_ingred'][i]) == 0:
                    lst.append(data)
            for word in copy_data['only_ingred'][i]:
                if data[0] in word:
                    lst.append(data)
    all_lst.append(lst)

100%|██████████| 113976/113976 [00:12<00:00, 9147.64it/s]


In [142]:
copy_data['ingredient'] = pd.Series(all_lst)

In [150]:
n = 83907
print(copy_data.iloc[n,]['ingredient'])
print(copy_data.iloc[n,]['only_ingred'])

[['손질 고등어', '1팩'], ['대파', '1대'], ['양파', '1/2개'], ['밀가루', '2T'], ['후추', '톡톡']]
['손질 고등어', '대파', '양파', '밀가루', '후추']


## 문자안에 특수문자 확인
- '-'
- '~'
- '/'
- '+'

### '-' 이 들어가 있는 데이터 확인

In [151]:
ingre_dash_set = set()
ingre_dash_index_set = set()
unit_dash_set = set()
unit_dash_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '-' in data[0]:
                ingre_dash_set.add(data[0])
                ingre_dash_index_set.add(i)
            if '-' in data[1]:
                unit_dash_set.add(data[1])
                unit_dash_index_set.add(i)
print(len(ingre_dash_set))
print(len(ingre_dash_index_set))
print(len(unit_dash_set))
print(len(unit_dash_index_set))  

100%|██████████| 113976/113976 [00:00<00:00, 140019.84it/s]

71
59
474
1491


#### 재료부분

In [152]:
for word in list(ingre_dash_set):
    print(word)

육우 - 홍두깨살
물 - 닭육수 농축액이 있을경우
냉동 가리비 조개의 패주/Sint-jakobsnoten
소금 약간 가나슈- 다크 커버춰
딸기바나나쉐이크-바나나
다슬기 무침용 -달걀
다슬기 무침용 -밀가루
깍둑썰기한 양파- 듬뿍
튀김옷 - 튀김가루
딸기잼 토핑용 약간 초코 사브레 - 버터
파이지 - 중력분
다슬기 무침용 -아욱
눈사람 - 버터
선택 - 양파
육수 - 물
육수 -
K-MILK 우유
생잎새버섯 2줌 양념장- 쑥갓 다진 것
라따뚜이 - 토마토
미니 바게트 - 우리밀 중력분
맛선생-해물맛내기
칵테일새우-중간이상사이즈
달걀 1개 -
물 150ml-
스모크햄 1개-
장식 - 멥쌀가루
얼음 탁구공만한것-! 큰덩어리얼음없음 각얼음 큰
송송 썬 파 / Lente-uitjes
라씨만들기-
미나리 2줌.멸치육수 600ml 등 양념장- 고추가루
쿠키바닥-버터
양념-된장
-조청
양파-채썬것
당근2개-썰어서 한줌 약간 넘게
체리청 - 체리
무우 한토막 - 무우채
김채 1줌 등 양념장- 진간장
육수 - 멸치
쿠키밑면 - 다이제 스티브
물 -만약 농축액이 없고 육수로만 할때 육수2리터+물1
쯔유-옵션
Semi-Dried 방울토마토
섬초 3줌 등 양념소스- 물
장식-달걀
꽈리고추 한봉- 약
휘밍크림-Whipping Cream 일인용기준
쫑쫑 썬 대파- 듬뿍
--버섯
베이컨 or 스팸 베이컨-
쿠키바닥-다이제
녹말가루-혹은 감자전분
돼지고기 안심-
계란 1개-
달걀물 - 달걀 노른자
애호박-생략가능
삶을 때 - 물
아이싱-달걀흰자
아이싱 - 슈거파우더
블루베리소 - 옥수수전분
알리오올리오 - 스파게티면
-참기름
생크림 -
도우 - 우리 밀가루강력분
물 1컵 양념장 - 두반장
아이싱- 생크림
닭육수의 농축액- Broth
옵션-블루베리콩포트
물 약간 등 양념장- 고추장
자연산기장미역-불린것
부추-썰어서


#### 재료부분처리

'-' 기준으로 나누고 앞에 있는 데이터를 사용하겠습니다.

In [153]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '-' in data[0]:
                data[0] = data[0].split('-')[0].strip()

100%|██████████| 113976/113976 [00:00<00:00, 149373.96it/s]


In [154]:
space_idx = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] == '':
                space_idx.append(i)

100%|██████████| 113976/113976 [00:00<00:00, 149707.68it/s]


In [155]:
for i in tqdm(space_idx):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] != '':
                lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/420 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/1768163441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 420/420 [00:00<00:00, 593.21it/s]


In [156]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] == '':
                print(data, i)

100%|██████████| 113976/113976 [00:00<00:00, 149771.56it/s]


#### 단위부분

In [157]:
for word in list(unit_dash_set):
    print(word)

2-3뿌리
1-1/4컵 
9-10
15-20개 정도
5개 =조선호박2-3개
1-2ts
12-15알
1-2작은 술 ​
5-6개 
3-5쪽
1-2숟갈
350-400g
2-3토막
40-45g
1-2t
2-3작은술
4-5개
80-100cc
4-5쪽
5-6개
5-6방울
2.5-3컵
1-2꼬집
3-4스푼
7-8개쯤
5-6알
3-4줄기
500-70ml
130-140g
30-40장
1/2-1모
65-70g
2-3잔
4-5줄
1-2개
1-1.5T
0.7-1T
2.5-3공기
6-8방울
2-3줄
1-2TBSP
2-3알
260g 소스 -물1/3컵
5-6줄기 송송썬것
1-2tsp
4-5쪽 or 양파 한개
7-8줄
4-500g
60-70g
4-5컵
4-5큰 술
7-8g
1/2-1마리
2-3컵
0.5-1모
1.5-2컵
1.5-2스푼
1.5-2작은 술
6-7장
2-3개 정도
1-1.5TS
4-8개
3-4쪽
1-2줄기
50-100ml
1/4-1/3분량
1-2쪽
8-9개
2-3숟가락
280-300g
9-10장
5-6컵
35-40개
15-20cm5개
1개 또는 방울토마토6-7개
7-8개
4-6장
3-4큰 술
1-2스푼
4-5조각
3-4큰술
216-220ml
3-1/2티스푼
7-8쪽
3-4큰술 등
0.5-1대
4-5바퀴
1-2kg
4-5스푼
20-25g
2-3잎
6-8장
13-15개
10-20g
3개-다지기
15-20마리
100-120ml
5-10ml
6-8마리
7-80g
4-5cm 정도
5-6마리
3-4컵 정도
2-3톨
1-2머그컵
3-4T
5-6슬라이스
60-70알
4-6개
20-30알
100-130g
10-20ml
1/2-1큰 술
0.5-1T
15-30장
100-150ml
50-60g
30-40ml
1-2줌
1-2큰술 등
600-700ml
1/2-1개
1-2대분 ​ ​ ​
8-9마리
7-10조각
22-23알
6-8줄기
4-5큰술
15-20장
5-6숟가락
1-2줄
400-500g
15-17알
3-4덩이
20-30g
3-4cm
6-10알
3-4토막
3-4줌
20-30개
1인당200-250g
1

단위쪽과 재료쪽의 구조가 조금 다른것 같습니다.

단위 부분은 '-' 뒤쪽만 살리도록 하겠습니다.

#### 단위부분처리

'-' 기준으로 나누고 뒤에 있는 데이터를 사용하겠습니다.

In [158]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '-' in data[1]:
                data[1] = data[1].split('-')[1].strip()

100%|██████████| 113976/113976 [00:00<00:00, 147065.92it/s]


In [159]:
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[1] != '':
                lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/113976 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/1261346209.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 113976/113976 [03:45<00:00, 505.70it/s]


In [160]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[1] == '':
                print(data, i)

100%|██████████| 113976/113976 [00:00<00:00, 148973.59it/s]


### '~' 들어가 있는 데이터 확인

In [161]:
ingre_wave_set = set()
ingre_wave_index_set = set()
unit_wave_set = set()
unit_wave_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '~' in data[0]:
                ingre_wave_set.add(data[0])
                ingre_wave_index_set.add(i)
            if '~' in data[1]:
                unit_wave_set.add(data[1])
                unit_wave_index_set.add(i)
                
print(len(ingre_wave_set))
print(len(ingre_wave_index_set))
print(len(unit_wave_set))
print(len(unit_wave_index_set)) 

100%|██████████| 113976/113976 [00:00<00:00, 137156.16it/s]

97
97
1548
7567


#### 재료부분

In [162]:
for word in list(ingre_wave_set):
    print(word)

생수 ~
돼지고기 앞다리살 150g~
3~4cm 두께의 식빵
4~5cm길이로 자른 부추
콩나물 반봉~
청양고추 3개~
통마늘 10개~
3~5시간 불린 찹쌀
쌀 1컵~
흰 설탕 800g~
고추장 1수저반~
얼린 딸기 8개~
데친 쑥 30g~
치킨스톡 4컵~
유자청 두 큰술 ~ 세
설탕 20g~
코코아 파우더 10g~
3~4cm길이로 자른 쪽파
5~6cm 길이로 자른 부추
대파 1개~
코코넛오일 60g~
2~3cm길이로 자른 부추
삶은 달걀 1개~
물 170ml~
차가운 물 10g~
물 1컵 반 ~
모짜렐라 치즈 140g ~
쌀조청 400g~
물 1컵반~
0세~2세까지 아기 멸치
500~600ml 병
닭고기 600g~
4~5cm 길이로 자른 달래
저지방 우유 170g ~
3~4cm길이로 자른 부추
바나나 큰거 1개~중간
훈제오리 250g ~
페페로치노 홀 7개~
푹~익은 배추김치
10~15cm길이로 자른 겨울초
5cm~6cm정도의 두께로 자른 식빵
4~5cm길이로 자른 쪽파
10~15cm 원형 사이즈
각종 전~
34~
카놀라유 400g~
검정깨 가루 30g~
물 100 g ~
10~15cm가량 다시마
통통한통삼겹~오겹♡
생땅콩 크게 세~네줌
비정제 설탕 80 g ~
마요네즈 2T~
전분 2T~
3~4cm 자른 부추
풋고추 1개~
익은김치~
우유 약 300ml ~
아몬드 밀가루 200g~
생연어&훈제연어 ~
멸치 8개~
김치 반포기~한
꿀 40 g ~
녹차가루 10g~
5~6cm 자른 부추
소금 생략가능 30cc~
과일~채소
냉이 1줌~
치즈~!
4~5cm자른 부추
찹쌀가루 ~
11~
소금 ~
묵은지김치 1포기~
카스테라~
요구르트 약 100ml~
물 ~
멸치육수 400ml~
소갈비 800g~
채끝살 100g~
콩나물 크~~게
양상추등~~
굴소스 2작은술~
우유 200ml~
청양고추 1개~
15~
레몬 10개 ~
설탕 1수저반~
밥 듬뿍~
물 40 g ~
라이스페이퍼 5장~
파슬리가루~
설탕 반컵 ~
5~6cm길이로 자른 부추
딸기 중~대형
오이 2개~
생크림요거트

#### 재료부분처리

'~'문자만 제거 해 주고, ' '(space)기준으로 나누어서 앞에 있는 데이터를 사용하겠습니다.

In [163]:
for i in list(ingre_wave_index_set):
    for data in copy_data['ingredient'][i]:
        if '~' in data[0]:
            data[0] = data[0].replace('~','')
            data[0] = data[0].split()[0]

In [164]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if '~' in data[0]:
            print(data)

100%|██████████| 113976/113976 [00:00<00:00, 154021.97it/s]


In [165]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data[0] == '':
            print(data)

100%|██████████| 113976/113976 [00:00<00:00, 156992.91it/s]


#### 단위부분

In [166]:
for word in list(unit_wave_set):
    print(word)

5~7장
1~2개
1~2아빠숟갈
20~22개
0.5~1개
1~개
15~17마리
130~140ml
2~2+1/2큰술
2~3TS
4~7큰술
5~7개
25~55g
3~4꼭지
1~2스픈
130~150ml
1~2톨
150~300g정도
하~안줌
30~60g
10~12알
7~10컵
10~13줄기
1/2컵~1컵
10~12kg
14~15개
90~100g 
4~5개
6~8줄
100~150cc
3~4송이
100~200g
0.5숟갈~1숟갈
3.5~4리터
50~60알
600~900g
1~2주걱
3~6줄
4~5국자
56~60개
1/2~1큰술 
2~5개
2~3센티한토막
6~8줄기
225~235g
2~3mm로 썬것11쪽
먹고싶은만큼적당히~
7~10개
4~5큰 술
먹고싶은만큼~
1큰술 ~ 입맛에 맞게
12~16개
1/2큰술 ~ 취향껏
160~180ml
0.3~0.4t
2컵~세컵
1/2컵~2/3컵
500~700g
50g ~ 취향껏
400~450g
9~10개정도
좋을듯요~
5~6줄기 정도
5~8쪽
8~9개
김밥갯수대로~
2~3주걱
450~500그람
5~10조각
600~700그람
2~3숟가락
2~3kg
1~2mg
70~80cc
4~5센티두께
6~7개
1/2~1/3티스푼
10~15cm1장
6~7장
60~65g
600~800g
115~135g
2/3~1공기
6~9장
3~4주먹
1/2~1t
2~8개
1~2마리 정도
13~15알
190~200g
입맛대로~
140~155g
90~110g
3~4웅큼
1.8~2리터
7~8T
1.5~2포기
취향대로~
110~120g
1+1/2~2T
5~6숟가락
10~15스푼
30~40정도
2/3~1개
2~3가지
2~3큰 술
1~2컵 사이
6~7장 정도
12~16알
200~210ml
2티백 /4~5g
600~700g
1/2~1/4알
5~6톨
1~1.5kg
5~6주먹
160~200g
150~200그람
2컵~2.5컵
160~170g
1~0.5국자
300~320cc
7~8뿌리
3~4그릇
220~240cc
2~2.5t
1~2통
10~15조각
25~30개
2~3g
30~40cc
3~4장

#### 단위부분처리

'~'기준으로 나누고 뒤에 있는 데이터를 사용합니다.

In [167]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if '~' in data[1]:
            data[1] = data[1].split('~')[1].strip()

100%|██████████| 113976/113976 [00:00<00:00, 155704.84it/s]


In [168]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if '~' in data[1]:
            print(data)

100%|██████████| 113976/113976 [00:00<00:00, 155281.01it/s]


In [169]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data[1] == '':
            print(data)

 42%|████▏     | 47900/113976 [00:00<00:00, 158712.26it/s]

['김', '']
['후추', '']
['파슬리가루', '']
['깻잎순', '']
['파마산 치즈가루', '']
['마요네즈', '']
['소금 OR 맛간장', '']
['치즈', '']
['버터', '']
['베이컨', '']
['달걀', '']
['다진 소고기 대략', '']
['비트 줄기', '']
['쌀뜨물', '']
['박력분 또는 중력분', '']
['깻잎', '']
['얼음', '']
['미나리', '']
['쯔란', '']
['대패 삼겹살', '']
['식용유', '']
['쌀뜨물', '']
['연유', '']
['뜨거운 물', '']
['소주', '']
['설탕', '']
['물', '']

 70%|██████▉   | 79462/113976 [00:00<00:00, 156319.02it/s]


['다시마물', '']
['튀김용 식용유', '']
['소금', '']
['간 돼지고기', '']
['카레가루', '']
['물', '']
['물', '']
['양파를 넣으셔도', '']
['햄', '']
['아위버섯', '']
['조금 달게 하시려면', '']
['맥주', '']
['중국식 만능 소스', '']
['생크림', '']
['생크림', '']
['피자 치즈', '']
['토핑용 크렌베리', '']
['굴소스', '']
['삶은 흰강낭콩', '']
['물', '']
['기름', '']
['건새우', '']
['등갈비', '']
['생강', '']
['오뎅', '']
['마요네즈', '']
['무', '']
['토마토소스', '']
['화이트소스', '']
['치킨', '']
['새우', '']
['올리고당', '']
['슬라이스치즈', '']
['슬라이스 햄', '']
['돼지등심', '']
['통모짜렐라치즈', '']
['소고기 샤브샤브용', '']
['통 모짜렐라치즈', '']
['빵가루', '']
['잡채', '']
['식초', '']
['유자청', '']
['레몬즙', '']
['옥수수전분가루', '']

100%|██████████| 113976/113976 [00:00<00:00, 156776.25it/s]


['설탕', '']
['팔팔 끓인 뜨거운 물', '']
['대패삼겹살', '']
['포도 최소', '']


In [170]:
#all_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[1] != '':
                lst.append(data)
    #all_lst.append(lst)
    copy_data['ingredient'][i] = lst

  0%|          | 0/113976 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/1261346209.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 113976/113976 [03:49<00:00, 496.92it/s]


In [171]:
#copy_data['ingredient'] = pd.Series(all_lst)

In [172]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data[1] == '':
            print(data)

100%|██████████| 113976/113976 [00:00<00:00, 158262.37it/s]


### '+' 들어가 있는 데이터 확인

In [173]:
ingre_plus_set = set()
ingre_plus_index_set = set()
unit_plus_set = set()
unit_plus_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                ingre_plus_set.add(data[0])
                ingre_plus_index_set.add(i)
            if '+' in data[1]:
                unit_plus_set.add(data[1])
                unit_plus_index_set.add(i)
print(len(ingre_plus_set))
print(len(ingre_plus_index_set))
print(len(unit_plus_set))
print(len(unit_plus_index_set))

100%|██████████| 113976/113976 [00:00<00:00, 138168.29it/s]

278
340
370
3343


#### 재료부분

In [174]:
for word in list(ingre_plus_set):
    print(word)

땅콩+아몬드
백앙금+초코시럽
들기름+참기름 각
배추김치+총각김치+고구마깍두기
빵가루+파슬리가루
+버터
호두+건 크랜베리
녹말물 전분가루 1 티스푼+따뜻한물
빵가루+허브가루
허니머스타드소스+레몬즙
계란+소금
들기름+식용유
미니 비타민+무순
불린 흰강낭콩과+불린 검은 콩
단무지+식초
박력분 210g+
굴소스 +물
깨소금+통깨
알+고니
핫케이크가루 2컵+
다시마+황태+파뿌리
흑설탕+황설탕
양파+호박
매생이+굵은소금
호두+아몬드
식초+레몬즙
알 +고니
체다+파마산치즈
백미7비율+현미3비율
올리브오일 1 Tbsp +
다시마+표고버섯 우린 물
우유 200ml +
우유 150ml +
지름 9 cm 디저트 컵 4개 + 16*11cm 그릇
양파+식촛물
고추장+스파게티소스
소금+간장
설탕 110g+
부침가루+튀김가루
우유+맛술
버터 150g +
시금치+당근+양파+청홍고추 약
밀가루+계란+빵가루
견과류 + 깨
황태육수황태+멸치+다시마
멸치액젓+국간장
현미+찰현미
쑥+쌀가루
된장 2큰술+
설탕+
모짜렐라+체다치즈
파슬리가루+훈제파프리카가루
들기름+올리브오일
감자전분+튀김가루
계란+우유
물5컵+
체다치즈 +모짜렐라치즈 약
전분물 전분가루 3T+물
치킨스톡1+물
올리브오일+마늘+쥐똥고추
올리브유 + 녹인버터
내장+알
오일+갓김치국물
밀가루 1컵 +
다시마+황태
블루베리+복분자
돈까스소스+머스타드소스
소금+후추+
김치 + 김치국물
버터+다진마늘
닭안심+닭닫리
비트 +사과즙간젓
비빔오징어젓갈+들기름
멸치+다시마
굵은소금+소주
소금+매실액+참기름
지름 15cm 높은 원형틀 한개분 + 머핀틀
멥쌀가루+찹쌀가루
배추 솎음+무 솎음
검은깨+참깨
달래+냉이
황태머리+천연조미료육수
국간장+멸치액젓 기호에
우유+생크림 각
소금+멸치액젓
카카오파우더+시나몬파우더
참치+마요네즈
비트 +사과즙 간 것
다진 돼지고기 +다진소고기
피클+할라피뇨
흑설탕+시나몬파우더
참기름+식용유
육수+물
소금+화이트페퍼
달걀노른자 3개+설탕
간장 + 고추냉이 소스
쌀+잡곡
화이트와인+레몬즙
깨소금 + 통깨
들기름+참기름
식초+

재료부분과 단위부분을 확인해보니  
문자의 맨앞과 맨뒤에 '+'가 붙어있는 것을 확인할 수 있었습니다.
제거해주도록 하겠습니다.

In [175]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                data[0] = data[0].strip('+')
            if '+' in data[1]:
                data[1] = data[1].strip('+')

100%|██████████| 113976/113976 [00:00<00:00, 139007.50it/s]


In [176]:
tmp = []
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                tmp.append(data[0])
print(len(tmp))

336


In [179]:
for word in tmp[:10]:
    print(word)

계란+우유
튀김가루+빵가루
밀가루+빵가루
삼색 하트파스타+삼색 동물모양 파스타
계란+우유
완두콩+팥배기
검은깨 + 참깨
빨강+ 노랑 파프리카
우유+달걀하나
백앙금+초코시럽


#### 재료부분처리

'+'가 들어가있는 재료는 제거하도록 하겠습니다.

In [180]:
all_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                pass
            else:
                lst.append(data)
    all_lst.append(lst)

100%|██████████| 113976/113976 [00:00<00:00, 125103.96it/s]


In [183]:
copy_data['ingredient'] = pd.Series(all_lst)

In [184]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[0]:
                print(data)

100%|██████████| 113976/113976 [00:00<00:00, 150961.47it/s]


#### 단위부분

'+'가 들어간 단위부분은 밑에서 처리할 '/'가 들어간 단위부분과 많이 겹쳐서 같이 처리하겠습니다.

In [187]:
plus_unit_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '+' in data[1]:
                plus_unit_lst.append(data[1])

100%|██████████| 113976/113976 [00:00<00:00, 152781.37it/s]


In [188]:
plus_unit_lst

['1+1/2개',
 '1+1/2컵',
 '2+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2T',
 '2+1/2개',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2줌',
 '1+1/2큰술',
 '2+1/2공기',
 '1+1/2공기',
 '1+1/2컵',
 '1+1/2개',
 '1+1/4컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2공기',
 '1+1/2공기',
 '2+1/2컵',
 '1+1/2컵',
 '1+1/2공기',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2장',
 '1+1/2개',
 '1+1/2마리',
 '1+1/2컵',
 '2+1/2컵',
 '1+1/2ts',
 '1+1/2T',
 '1+1/2공기',
 '1+1/2공기',
 '1+1/2컵',
 '1+1/2큰술',
 '1+1/2컵',
 '3+1/2컵',
 '2+1/2밥공기',
 '2+1/2종이컵',
 '1+1/2컵',
 '1+1/2컵',
 '2+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2공기',
 '3+1/2개',
 '1+1/2컵 150g',
 '2+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2개',
 '1+1/2공기',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2컵',
 '2+1/2컵',
 '1+1/2숟갈',
 '1+1/2개',
 '4+1/2장',
 '1+1/2술',
 '2+1/2컵',
 '2+1/2숟가락',
 '1+1/2컵',
 '1+1/2컵',
 '1+1/2작은술',
 '1+1/2큰술',
 '2+1/2컵',
 '1+1/2공기',
 '2+1/2컵',
 '1+1/2컵',
 '1+1/2큰술',
 '4+1/2컵',
 '1+1/2개',
 '1+1/2큰술',
 '2+1/2컵',
 '1+1/2개',
 '1+1/2개',
 '1+1/2줌',
 '1+1/2줌',
 '2+1/2컵',
 '6+1/2개',
 '1+1/2공기',
 '3+1/2컵',


###  '/' 들어가 있는 데이터 확인

In [189]:
ingre_slash_set = set()
ingre_slash_index_set = set()
unit_slash_set = set()
unit_slash_index_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '/' in data[0]:
                ingre_slash_set.add(data[0])
                ingre_slash_index_set.add(i)
            if '/' in data[1]:
                unit_slash_set.add(data[1])
                unit_slash_index_set.add(i)
print(len(ingre_slash_set))
print(len(ingre_slash_index_set))
print(len(unit_slash_set))
print(len(unit_slash_index_set))

100%|██████████| 113976/113976 [00:00<00:00, 134564.74it/s]

382
443
1605
49641


#### 재료부분

In [190]:
for word in list(ingre_slash_set):
    print(word)

전분가루 3T/
빨강/주황 파프리카
배합초/유부초밥소스
식초 / 중국백식초
목살/삼겹살
핫칠리소스/스위트칠리소스
쪽파/잔파 파란부분
우유/물
당면/소면
청주/고량주
밀가루 /전분가루
매실액/유자소스
남은치킨/닭가슴살큐브or닭가슴살
SoJu Aardbei smaak/딸기맛 소주
파프리카/양파
노랑/주황 파프리카
레몬즙/레몬주스
꿀/조청시럽
당근/부추합쳐서
진간장/쯔유
고추기름 /고춧가루
다시육수/물
부추 / 쪽파
닭육수/채수/생수
소금 / 후추 / 올리브오일
꿀/매실액
꿀/설탕
아보카도오일 / 올리브오일
피망 /당근 다진것
햄/베이컨
양파/과일/견과류 자유롭게
삶은달걀/삶은메추리알
심미유/식용유
크림치즈/훼타치즈
마늘 5개/ 다진마늘
노랑/주황/빨강 파프리카
해바라기씨/아몬드/호박씨
계란과자/다이제
물엿/꿀/아가베 납작한 국자로
아몬드슬라이스/견과류
오레오/깜뜨
단호박/늙은호박
멸치육수/천연조미료육수
돼지고기 앞다리살/삼겹살
스파게티/앤젤헤어
건크랜베리/건포도
자반/조미김
식품건조기/에어프라이어
액젓/피시소스
데코용야채/밥
청양고추/세라노고추/할라피뇨고추
식용유/버터
네임팬/유성팬
육회/육사시미용소고기
파프라카/당근
새조개/해물
맛간장/진간장
방울토마토 노랑/빨강
화이트/블랙초콜릿
닭/돼지고기/오리/소고기
마가린/버터/식용유
다진 마늘 /
모짜렐라/체다/고다치즈믹스
소금 /
조청/메이플시럽
송송 썬 파 / Lente
설탕 50g/
햄 /쏘이팜
멸치육수/다시마육수/천연조미료육수/황태육수
청양/홍고추 각
1/2빵 팬
양배추/배추/깻잎/부추
흑초/식초
이스트 5g/
Tonic/ 토닉 워터
참기름/들기름
실온/데운 우유
로메인/상추/양상추등
1/8크기의조미김
딸기시럽/딸기잼
물엿/꿀
식물성 오일/올리브유 제외
녹인 버터/무가염 버터
1/ 2빵팬
모닝빵/식빵
토마토소스/브라바소스
청주/소주/보드카등
1/4통 양배추
송이/표고/팽이/느타리버섯
들기름/참기름
적/황 파프리카
햄/소시지
녹인 버터 / 코코넛 오일
종이/호일
소금/새우젓
대파/미나리/부추
베이컨/삼겹살
생강술/

#### 재료부분처리

'/'를 기준으로 두번째 데이터를 가져옵니다.

In [191]:
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '/' in data[0]:
                data[0] = data[0].split('/')[1].strip()

In [192]:
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if '/' in data[0]:
                print(data)

In [193]:
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        if data != []:
            if data[0] != '':
                lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/113976 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/1993634024.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 113976/113976 [03:34<00:00, 531.12it/s]


In [194]:
for i in range(len(copy_data['ingredient'])):
    for data in copy_data['ingredient'][i]:
        if data != []:
            if  data[0] == '':
                print(data)

#### 단위부분

In [195]:
for word in list(unit_slash_set):
    print(word)

1cup &1/2cup
2+1/2리터
1+1/2숟가라
1.1/2종이컵
1/3어른 숟가락
8/1개
1+1/2큰 술
3/2컵
1/2공기분량
1/2모
1개 양파1/4
1/2통정도
1/2개 정도
1/4개 또는 한줌
1/3가량 
1/8대
1모/팩
3/4
1/2통에서한통
1/4cc
2+3/4컵
1+1/2국그릇
1/2큰술 소떡소떡 소스 사과1개
1/4쪽 보다 적게
2/3국그릇
2+1/2근
1/2t + 약간
1티스푼+2/3티스푼
1/5모
1/2봉지
1/2반
5cm1/3개
6+1/2컵
2/1숟가락
1/2g
4/5T
1/2cc
1/2개100g
3/4마리
1/2컵*
1/6도막
1/2숟갈
1/2인분
1/2스폰
1/3도막
1+1/2뿌리
1/2모 또는 연두부
1/8tsp
3/1티스푼
2머그컵1/2컵
1/5개=두께3cm정도
1/2머그컵
1/2개80g
1/2T 설탕1/2T
2+3/4Cup
3/1정도
1/1t
1/2스푼 적당량
1/4개분
2/4ts
1/3만
1/4모110g
1+1/2컵 
1/2틀 한개분
1/2큰술+물1큰술
1/8숟가락
1컵+1/3T
1개 양파1/4개
1/4만 썰기
1/3봉
1공기+1/2공기
2/3개 대파1개
1/2단
1+1/2봉지
1/4스푼
2/3개
3/4T
1/2개 양송이버섯2개
2/5작은술
1/2Tbsp
1/3작 은 술
1/2그릇
1+1/2L
1/2컵1T
1/2개분량
1개+2/3개
3+1/2줌
1/2T정도
1/6포기
1/2움큼
1or1/2T
2+1/2포기
1+1/2공기
1/3컵씩
2/3큰술+1작은술
3/4컵
1/3스푼
1/10개=두께1.5cm정도
1/2컵적당량
1/2개 물1/2컵
2/3장
1/2찻숟가락
1/3인분
1/2작은술+약간
1/3컵+1/2컵
1+1/2컵정도
4/5큰 술
1+1/2움큼
1/3tsp
1/4작은술 후춧가루1/4작은술
2/1모
1/4개정도
3/4개 or1개
1+1/2컵 400ml
11/2공기
1/2Tsp
3컵+1/2컵
1/2컵보다조금더
3/4팩
2컵+1/2컵
1/2죽
3/4ts
1+1/2손
2/3등분
1/8도막
1+1/2주걱
집게/젓가락
1+1/2Ts
1

#### 단위부분처리

단위부분을 처리하는 함수는 다른 특수문자를 포함한 데이터를 처리한뒤에 수행하겠습니다.

In [196]:
import re
def func(word):
    word = word.split()[0]
    lst = []
    check = True
    for i in range(len(word)):
        try:
            int(word[i])
            break
        except:
            check = False
    if check:
        for i in range(len(word)):
            try:
                v = round(eval(word[:i+1]),1)
                lst.append(v)
            except:
                pass
        special = re.compile(r'[^ A-Za-z가-힣]')
        special2 = re.compile(r'[^ .0-9]')
        result = str(lst[-1]) + str(special.sub('', word))
        value = special2.sub('', result)
        unit = special.sub('', result)
        return [value, unit]
    else:
        return ['1', '단위']

In [220]:
 copy_data['ingredient'][12222]

[['스파게티면', '160g'],
 ['명란젓 큰 것', '1개'],
 ['우유', '1+1/2컵'],
 ['생크림', '1+1/2컵'],
 ['건고추', '3개'],
 ['마늘', '5쪽'],
 ['올리브유', '2큰술'],
 ['구운김', '약간']]

In [221]:
for data in copy_data['ingredient'][12222]:
    word = data[1]
    print(word)
    print(func(word))
    print('-'*10)

160g
['160', 'g']
----------
1개
['1', '개']
----------
1+1/2컵
['1.5', '컵']
----------
1+1/2컵
['1.5', '컵']
----------
3개
['3', '개']
----------
5쪽
['5', '쪽']
----------
2큰술
['2', '큰술']
----------
약간
['1', '단위']
----------


In [414]:
lst =[['당근','1개'], ['양파', '2개']]

for i in range(len(lst)):
    temp = []
    temp.append(lst[i][0])
    temp += func(lst[i][1])
    lst[i] = temp
lst

[['당근', '1', '개'], ['양파', '2', '개']]

### 우리가 설정한 특수문자외 다른 특수문자 제거

In [263]:
test ='!!ada'
result = re.sub(r'[^-~+/.0-9a-zA-Z가-힣 ]', '',test)
result

'ada'

In [265]:
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            data[0] = re.sub(r'[^-~+/.0-9a-zA-Z가-힣 ]', '', data[0]).strip()
            data[1] = re.sub(r'[^-~+/.0-9a-zA-Z가-힣 ]', '', data[1]).strip()

100%|██████████| 113976/113976 [00:02<00:00, 49647.52it/s]


In [267]:
special = re.compile(r'[-~+/.0-9a-zA-Z가-힣 ]')
result = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data != []:
            result.add(special.sub('', data[0]))
            result.add(special.sub('', data[1]))

100%|██████████| 113976/113976 [00:01<00:00, 60874.16it/s]


### 마지막으로 공백문자 혹은 빈칸 확인

In [270]:
drop_index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        if data == []:
            drop_index.append(i)

100%|██████████| 113976/113976 [00:00<00:00, 167120.49it/s]


In [271]:
len(drop_index)

0

In [272]:
index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
            if data[0] =='' or data[1] =='':
                index.add(i)

100%|██████████| 113976/113976 [00:00<00:00, 150959.85it/s]


In [278]:
for i in tqdm(index):
    lst =[]
    for data in copy_data['ingredient'][i]:
        if data[0] == '' or data[1] == '':
            pass
        else:
            lst.append(data)
    copy_data['ingredient'][i] = lst

  0%|          | 0/84 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp/ipykernel_15320/2704427141.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_data['ingredient'][i] = lst
100%|██████████| 84/84 [00:00<00:00, 480.00it/s]


In [286]:
drop_index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    if copy_data['ingredient'][i] == []:
        drop_index.add(i)

100%|██████████| 113976/113976 [00:00<00:00, 188079.22it/s]


In [293]:
copy_data = copy_data.drop(list(drop_index))
copy_data = copy_data.reset_index(drop=True)
copy_data.head()

,id,ingredient,only_ingred
0,128671,"[[어묵, 2개], [김밥용김, 3장], [당면, 1움큼], [양파, 1/2개], ...","[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,131871,"[[현미, 4컵], [찹쌀, 2컵], [호두, 50g], [물, 1/2컵], [소금...","[현미, 찹쌀, 호두, 물, 소금]"
2,139247,"[[북어포, 1마리], [찹쌀가루, 1C]]","[북어포, 찹쌀가루]"
3,149207,"[[파스타면, 1단위]]",[파스타면]
4,151148,"[[건표고버섯, 9개], [오이, 1/2개], [당근, 1/2개], [양파, 1/2...","[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


In [294]:
drop_index = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    if copy_data['ingredient'][i] == []:
        drop_index.add(i)

100%|██████████| 113772/113772 [00:00<00:00, 181166.22it/s]


In [295]:
len(drop_index)

0

### 위에서 언급했던 단위부분처리('+','/')하기

In [296]:
lst =[['당근','1개'], ['양파', '2개']]

for i in range(len(lst)):
    temp = []
    temp.append(lst[i][0])
    temp += func(lst[i][1])
    lst[i] = temp
lst

[['당근', '1', '개'], ['양파', '2', '개']]

In [297]:
all_lst = []
for i in tqdm(range(len(copy_data['ingredient']))):
    lst = []
    for data in copy_data['ingredient'][i]:
        temp = []
        temp.append(data[0])
        temp += func(data[1])
        lst.append(temp)
    all_lst.append(lst)

100%|██████████| 113772/113772 [00:16<00:00, 6761.67it/s]


In [300]:
copy_data['ingredient'] = pd.Series(all_lst)
copy_data['ingredient']

0         [[어묵, 2, 개], [김밥용김, 3, 장], [당면, 1, 움큼], [양파, 0...
1         [[현미, 4, 컵], [찹쌀, 2, 컵], [호두, 50, g], [물, 0.5,...
2                              [[북어포, 1, 마리], [찹쌀가루, 1, C]]
3                                           [[파스타면, 1, 단위]]
4         [[건표고버섯, 9, 개], [오이, 0.5, 개], [당근, 0.5, 개], [양...
                                ...                        
113767    [[양배추, 0.2, 통], [양파, 0.5, 개], [편마늘, 1, 숟가락], [...
113768    [[박력분, 200, g], [버터, 70, g], [크림치즈, 70, g], [설...
113769    [[포항초, 300, g], [다시멸치, 1, 줌], [홍고추, 1, 개], [대파...
113770    [[식빵, 4, 장], [감자, 2, 개], [양파, 0.2, 개], [피망, 0....
113771              [[배추잎, 10, 장], [홍고추, 1, 개], [계란, 2, 개]]
Name: ingredient, Length: 113772, dtype: object

## 단위 통일(종이컵 or cup ->컵)

어떤 단위가 존재하는 지 확인해보겠습니다.

In [301]:
unit_set = set()
for i in tqdm(range(len(copy_data['ingredient']))):
    for data in copy_data['ingredient'][i]:
        unit_set.add(data[2])

100%|██████████| 113772/113772 [00:00<00:00, 144932.69it/s]


In [302]:
len(unit_set)

1429

In [303]:
for word in list(unit_set):
    print(word)


쪽쪽
컵T
중
톤
개분
EA
믂음
속
큰술이하
개이상
TS듬뿍
g이상
종류
통분량
스푼인대
대대
손가득
그램정도
숟
돈
or개
그릇그릇
cm정도썰은것
번톡
웅쿰
T큰술
숟갈정도
세젓가락분
차숟갈
어른숟가락
면
원사이즈로
쪽분
개정도
토박
컨
작은캔
병
cm길이토막
줌정
모반
ml
다발
PCS
덩이반
대반
웅큼씩
춤큼
g허브솔트작은술
대
백원동전만한
작은줌
t정도
대분
그릇분
L정도
세줌
O개
인분짜리
팩씩
인분양
TS
큰술분
약지손가락길이
톨가량
종이컵꽉채워
zjq
t반
봉반
줌
섯개
무더기
g반개
g짜리
장반
Ts듬뿍
큰밥숟가락
Tbs소금
센티길이
김
폄
움큼
컴
컵분량
개Kg
kgs
tsts
그릇이상
개는
국자듬뿍
밥숟가락크게
KG
T분량
x
g남짓
봉
아이스크림쿠퍼
솥
스푼씩
꼬집식
수저아빠숟갈
스푼가량
잔분량
마리이상
개틀
웅큼정도
가지쪽씩
검지손가락
펙
세방울
Tsps
컬러
CC
큰술슿
컵보다작게
공기가득
긴줄기
사이즈
부피정도
우동대접
호
조각씩
꼬지집
댜
종이컵가득
바퀴
원동전
숟갈락
단kg
t만
쌍
티스푸
찻술
컵조금더
푼
tT
개남짓
큰술이내
수북한
쪽내외
재기반
어장
개식
줄분량
개량
ml씩
동
종지
종이컵g
티스푼티스푼
ml분량
mll
줌반
CM
마리내외
종세트
밤양만큼
장가량
줌컵
용기
냄비기준
gtbsp
큰숟가락
리터개
캅반
밥숟갈
작
덩어리
필렛
밥스푼
개만들양만큼
두름
척용
원크기
타래
Kg
푸리
스푼밥수저기준
꼬집정도
알쯤
한줌
만원
kg내외
큰술반
가닥정도
작은술름
뿌리내외
cm길이로토막
시간불린
파리
모G
T과T
kg개
cm크기
댓조각
ML이상
Tds
호로
주먹의
뿌리분량
숨
g가량
공기밥그릇
꼬
스푼이하
컵가량
국컵
봉지반
줌분량
인분의
cups
과작은술
컵씩
종이컵꽉채워서
스푼정도
Cm
배정도
주먹씩
뿌리정도
겹
gg
g껍질제외
모
밥술
컵인데
ml정도
숟갈한꼬집
컵정도
조각통째로
set
섯숟가락
동전개
달걀잠길정도
장가쪽짜뚜리
댓장
롤
cm조각
수북하게
과공기
통
캐
모개
개푼물
쪽그램
냉면그릇
덩이정도
짜리동전만큼
정


단위를 변경할 재료 리스트입니다.

In [318]:
def count_unit(word):
    unit_list = []
    for i in range(len(copy_data['ingredient'])):
        for data in copy_data['ingredient'][i]:
            if data[0] == word:
                unit = data[2]
                unit_list.append(unit)
    counter = Counter(unit_list)
    print(word + ":", end=' ')
    print(counter.most_common(3))

In [328]:
ingredient_list = ['감귤','귤','김','팥','방울토마토','포도','쇠고기','소고기','팥','쌀','찹쌀','콩','녹두','고구마','감자','배추','양배추','상추',
                   '오이','호박','토마토','딸기','무','당근','건고추','풋고추','붉은고추','마늘','양파','파','생강','미나리',
                   '깻잎','피망','파프리카','멜론','땅콩','느타리버섯','팽이버섯','새송이버섯','사과','배','단감','바나나',
                   '돼지고기','닭고기','계란','달걀','명태','건멸치','건오징어','건미역','굴','조기','호두','아몬드',
                   '레몬','건포도','파인애플','새우','김치','즉석밥','두부','맛김','조미김','콩나물','얼갈이배추','수박','우유','고등어',
                   '건블루베리','참다래','전복','꽁치','시금치','열무','오렌지','갈치','물오징어','오징어','망고','가지','부추','참외',
                   '아보카도','체리','복숭아'
                    ]
for ingredient in ingredient_list:
    count_unit(ingredient)

감귤: [('개', 12), ('단위', 2), ('g', 2)]
귤: [('개', 142), ('단위', 54), ('g', 11)]
김: [('장', 892), ('단위', 601), ('봉지', 14)]
팥: [('컵', 76), ('g', 60), ('단위', 46)]
방울토마토: [('개', 746), ('단위', 255), ('알', 72)]
포도: [('송이', 45), ('단위', 32), ('알', 18)]
쇠고기: [('g', 297), ('단위', 32), ('줌', 7)]
소고기: [('g', 1612), ('단위', 327), ('줌', 63)]
팥: [('컵', 76), ('g', 60), ('단위', 46)]
쌀: [('컵', 600), ('단위', 146), ('인분', 135)]
찹쌀: [('컵', 358), ('단위', 96), ('g', 60)]
콩: [('컵', 25), ('단위', 22), ('g', 11)]
녹두: [('컵', 26), ('단위', 18), ('g', 14)]
고구마: [('개', 933), ('단위', 243), ('g', 152)]
감자: [('개', 5503), ('단위', 744), ('g', 245)]
배추: [('포기', 152), ('장', 151), ('단위', 135)]
양배추: [('단위', 1035), ('g', 540), ('개', 484)]
상추: [('장', 568), ('단위', 360), ('g', 55)]
오이: [('개', 3883), ('단위', 975), ('g', 83)]
호박: [('개', 867), ('단위', 597), ('g', 104)]
토마토: [('개', 1340), ('단위', 369), ('g', 45)]
딸기: [('개', 381), ('단위', 198), ('g', 74)]
무: [('개', 1620), ('g', 1086), ('단위', 903)]
당근: [('개', 6745), ('단위', 4597), ('g', 1085)]
건고추: [('개',

In [330]:
ingre_dict = {
            '감귤': '개',
            '귤': '개',
            '김': '장',
            '팥': '컵',
            '방울토마토': '개',
            '포도': '송이',
            '쇠고기': 'g',
            '소고기': 'g',
            '팥': '컵',
            '쌀': '컵',
            '찹쌀': '컵',
            '콩': '컵', 
            '녹두': '컵',
            '고구마':'개',
            '감자': '개',
            '배추': '포기',
            '양배추': '포기',
            '상추': '장',
            '오이': '개',
            '호박': '개',
            '토마토': '개',
            '딸기': '개',
            '무': '개',
            '당근': '개',
            '건고추': '개',
            '풋고추': '개',
            '붉은고추': '개',
            '마늘': '개',
            '양파': '개',
            '파': '개',
            '생강': '개',
            '미나리': 'g',
            '깻잎': '장',
            '피망': '개',
            '파프리카': '개',
            '멜론': '개',
            '땅콩': '개', 
            '느타리버섯': '개',
            '팽이버섯': '봉지',
            '새송이버섯': '개',
            '사과': '개',
            '배': '개',
            '단감': '개',
            '바나나': '개',
            '돼지고기': 'g',
            '닭고기': 'g',
            '계란': '개',
            '달걀': '개',
            '명태': '마리',
            '건멸치': 'g',
            '건오징어': '마리',
            '건미역': 'g', 
            '굴': 'g',
            '조기': '마리',
            '호두': 'g',
            '아몬드': 'g',
            '레몬': '개',
            '건포도': 'g',
            '파인애플': '개',
            '새우': '마리',
            '김치': '포기',
            '즉석밥': '개',
            '두부': '모',
            '맛김': '장',
            '조미김': '장',
            '콩나물': 'g',
            '얼갈이배추': '단',
            '수박': '개',
            '우유': 'ml',
            '고등어': '마리',
            '건블루베리': 'g',
            '참다래': '개',
            '전복': '마리',
            '꽁치': '마리',
            '시금치': '단',
            '열무': '단',
            '오렌지': '개',
            '갈치': '마리',
            '물오징어': '마리',
            '오징어': '마리',
            '망고': '개',
            '가지': '개',
            '부추': 'g',
            '참외': '개',
            '아보카도': '개',
            '체리': '개',
            '복숭아': '개'
              }

In [333]:
def change_unit(word):
    for i in range(len(copy_data['ingredient'])):
        for data in copy_data['ingredient'][i]:
            if data[0] == word:
                data[2] = ingre_dict[word]

In [335]:
for ingredient in tqdm(ingredient_list):
    change_unit(ingredient)

100%|██████████| 87/87 [00:57<00:00,  1.52it/s]


In [336]:
for ingredient in ingredient_list:
    count_unit(ingredient)

감귤: [('개', 16)]
귤: [('개', 224)]
김: [('장', 1552)]
팥: [('컵', 212)]
방울토마토: [('개', 1195)]
포도: [('송이', 120)]
쇠고기: [('g', 357)]
소고기: [('g', 2210)]
팥: [('컵', 212)]
쌀: [('컵', 1156)]
찹쌀: [('컵', 616)]
콩: [('컵', 67)]
녹두: [('컵', 67)]
고구마: [('개', 1353)]
감자: [('개', 6739)]
배추: [('포기', 749)]
양배추: [('포기', 3590)]
상추: [('장', 1123)]
오이: [('개', 5094)]
호박: [('개', 1707)]
토마토: [('개', 1828)]
딸기: [('개', 792)]
무: [('개', 4954)]
당근: [('개', 13513)]
건고추: [('개', 243)]
풋고추: [('개', 1034)]
붉은고추: [('개', 137)]
마늘: [('개', 8120)]
양파: [('개', 32621)]
파: [('개', 6851)]
생강: [('개', 1669)]
미나리: [('g', 1013)]
깻잎: [('장', 3443)]
피망: [('개', 1201)]
파프리카: [('개', 3261)]
멜론: [('개', 23)]
땅콩: [('개', 266)]
느타리버섯: [('개', 1578)]
팽이버섯: [('봉지', 2017)]
새송이버섯: [('개', 2142)]
사과: [('개', 1770)]
배: [('개', 1016)]
단감: [('개', 107)]
바나나: [('개', 861)]
돼지고기: [('g', 2301)]
닭고기: [('g', 289)]
계란: [('개', 12316)]
달걀: [('개', 9250)]
명태: [('마리', 8)]
건멸치: [('g', 18)]
건오징어: [('마리', 9)]
건미역: [('g', 203)]
굴: [('g', 765)]
조기: [('마리', 59)]
호두: [('g', 904)]
아몬드: [('g', 48

## 추천 시스템에 사용할 수 있게 데이터프레임 만들기

In [339]:
import pandas as pd

In [340]:
data = {"일련번호" :[123123, 345335, 53424],
        "재료" :  [[["양파", "2개"],['양파','2개'], ["당근", '3개'], ['밀가루', '100g']],[["브로콜리", '1개'], ['물', '1+1/2컵'], ['상추', '8~10장']], []]}

df = pd.DataFrame(data)
df

,일련번호,재료
0,123123,"[[양파, 2개], [양파, 2개], [당근, 3개], [밀가루, 100g]]"
1,345335,"[[브로콜리, 1개], [물, 1+1/2컵], [상추, 8~10장]]"
2,53424,[]


In [347]:
copy_data.head()

,id,ingredient,only_ingred
0,128671,"[[어묵, 2, 개], [김밥용김, 3, 장], [당면, 1, 움큼], [양파, 0...","[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,131871,"[[현미, 4, 컵], [찹쌀, 2, 컵], [호두, 50, g], [물, 0.5,...","[현미, 찹쌀, 호두, 물, 소금]"
2,139247,"[[북어포, 1, 마리], [찹쌀가루, 1, C]]","[북어포, 찹쌀가루]"
3,149207,"[[파스타면, 1, 단위]]",[파스타면]
4,151148,"[[건표고버섯, 9, 개], [오이, 0.5, 개], [당근, 0.5, 개], [양...","[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"


In [349]:
ingredient_set = set()
for li in tqdm(copy_data['ingredient']):
    for data in li:
        ingredient_set.add(data[0])

100%|██████████| 113772/113772 [00:00<00:00, 476038.53it/s]


In [352]:
len(ingredient_set)

36787

In [341]:
# ingredient_set = set()
# for li in df['재료']:
#   for data in li:
#     ingredient_set.add(data[0])

# ingredient_set

{'당근', '물', '밀가루', '브로콜리', '상추', '양파'}

In [342]:
# numbers_dic = dict()
# for i in range(len(df)):
#   index = df.iloc[i, 0]
#   lst =[]
#   for li in df.loc[(df['일련번호']==index)]['재료']:
#     for data in li:
#       lst.append({data[0]:data[1]})
#   numbers_dic[index] = lst
# numbers_dic

{123123: [{'양파': '2개'}, {'양파': '2개'}, {'당근': '3개'}, {'밀가루': '100g'}],
 345335: [{'브로콜리': '1개'}, {'물': '1+1/2컵'}, {'상추': '8~10장'}],
 53424: []}

In [353]:
numbers_dic = dict()
for i in tqdm(range(len(copy_data))):
    index = copy_data.iloc[i, 0]
    lst =[]
    for li in copy_data.loc[(copy_data['id']==index)]['ingredient']:
        for data in li:
            lst.append({data[0]:data[1]})
    numbers_dic[index] = lst
numbers_dic

100%|██████████| 113772/113772 [00:52<00:00, 2173.96it/s]


{128671: [{'어묵': '2'},
  {'김밥용김': '3'},
  {'당면': '1'},
  {'양파': '0.5'},
  {'당근': '0.5'},
  {'깻잎': '6'},
  {'튀김가루': '1'},
  {'올리브유': '1'},
  {'간장': '1'},
  {'참기름': '1'}],
 131871: [{'현미': '4'}, {'찹쌀': '2'}, {'호두': '50'}, {'물': '0.5'}, {'소금': '1'}],
 139247: [{'북어포': '1'}, {'찹쌀가루': '1'}],
 149207: [{'파스타면': '1'}],
 151148: [{'건표고버섯': '9'},
  {'오이': '0.5'},
  {'당근': '0.5'},
  {'양파': '0.5'},
  {'사과': '0.5'},
  {'그외의': '1'},
  {'과일': '1'}],
 153040: [{'닭': '1'}],
 221094: [{'양파': '1'}, {'빵가루': '1'}, {'밀가루': '1'}, {'물': '1'}, {'파슬리': '1'}],
 221097: [{'밥': '2'},
  {'밥': '2'},
  {'참치캔 큰거': '1'},
  {'양파': '1'},
  {'다진김치': '1'},
  {'김밥용김': '1'}],
 221098: [{'떡볶이떡': '150'},
  {'당근': '0.5'},
  {'파': '1'},
  {'파': '1'},
  {'양파': '0.5'},
  {'양배추': '0.2'},
  {'어묵': '2'},
  {'오징어': '1'},
  {'새우': '1'},
  {'조갯살': '1'},
  {'홍합살': '1'}],
 221101: [{'스팸 싱글': '1'}, {'토마토': '1'}, {'상추': '2'}, {'치즈': '1'}],
 221102: [{'밥': '2'}, {'깻잎': '10'}, {'참치 작은캔': '1'}, {'당근': '0.5'}],
 223583: [{'새우': '10'},
  {'밥': 

In [343]:
# new = pd.concat([df['일련번호'], pd.DataFrame(columns=ingredient_set)], axis=1)
# new

,일련번호,상추,물,양파,당근,밀가루,브로콜리
0,123123,NaN,NaN,NaN,NaN,NaN,NaN
1,345335,NaN,NaN,NaN,NaN,NaN,NaN
2,53424,NaN,NaN,NaN,NaN,NaN,NaN


In [354]:
new = pd.concat([copy_data['id'], pd.DataFrame(columns=ingredient_set)], axis=1)
new.head()

,id,돼지고기갈비,소고기 양지나 치마살,캐슈너트,큐원 홈메이드 고구마케익믹스,참치 통조림 150g,닭다리 삶을,재료 사과,대파 10cm 길이,삶은 땅콩 깐것,...,구연산,건포도 적당량,소시지햄,유카,시판되는 사골국물,갈아둔 돼지고기,말린 황태,콩나물김치국,열무데칠때 소금,샌드위치용사각햄
0,128671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,131871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,139247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,149207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,151148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [356]:
for i in tqdm(range(len(copy_data))):
    index = copy_data.iloc[i, 0]
    for ingredient in list(ingredient_set):
        for li in numbers_dic[index]:
            try:
                value=li[ingredient]
                new.loc[(new['id']==index),ingredient]=float(value)
            except:
                pass

new = new.fillna(0)
new.head()

100%|██████████| 113772/113772 [2:41:49<00:00, 11.72it/s] 


,id,돼지고기갈비,소고기 양지나 치마살,캐슈너트,큐원 홈메이드 고구마케익믹스,참치 통조림 150g,닭다리 삶을,재료 사과,대파 10cm 길이,삶은 땅콩 깐것,...,구연산,건포도 적당량,소시지햄,유카,시판되는 사골국물,갈아둔 돼지고기,말린 황태,콩나물김치국,열무데칠때 소금,샌드위치용사각햄
0,128671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,131871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,139247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,149207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,151148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 다른 곳에서 사용할 수 있도록 저장

In [358]:
# new.to_csv('modeling_text_data.csv', index=False)

In [360]:
copy_data.to_parquet('ingredient_text_preprocessing.parquet',index=False)

In [364]:
new.to_pickle('modeling_text_data.pkl')

In [368]:
# load_file = pd.read_pickle('modeling_text_data.pkl')
# load_file.head()

In [371]:
# load_data = pd.read_parquet('ingredient_text_preprocessing.parquet')
# load_data.head()

### 코사인 유사도 구하기

In [377]:

half = len(new)//2
new_copy = new.iloc[:half+1, ]
new_copy.shape

(56887, 36788)

In [379]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(new_copy)
sacled_df

MemoryError: Unable to allocate 15.6 GiB for an array with shape (36787, 56887) and data type float64